In [ ]:
# Save all badges to DynamoDB

import boto3
import pandas as pd
import requests
import os
from datetime import datetime

# Configure AWS credentials
aws_access_key_id = os.environ.get('AWS_ACCESS_KEY')
aws_secret_access_key = os.environ.get('AWS_SECRET_KEY')
aws_region = os.environ.get('AWS_REGION')
aws_table = os.environ.get('AWS_TABLE')

# Create a DynamoDB resource
dynamodb = boto3.resource(
  'dynamodb', 
  aws_access_key_id=aws_access_key_id,                        
  aws_secret_access_key=aws_secret_access_key,
  region_name=aws_region
)

# Configure Twitch credentials
client_id = os.environ.get('TWITCH_CLIENT_ID')
client_secret = os.environ.get('TWITCH_CLIENT_SECRET')
oauth = f"https://id.twitch.tv/oauth2/token?client_id={client_id}&client_secret={client_secret}&grant_type=client_credentials"

# Get the DynamoDB table
table = dynamodb.Table(aws_table)

response = requests.post(oauth)

# Check the response
if response.status_code == 200:
  # Successful request

  # Set your environmental variable
  api_key = response.json()["access_token"]

  # Construct headers with the required information
  headers = {
    'Authorization': f'Bearer {api_key}',
    'Client-Id': client_id
  }

  # API endpoint URL
  url = 'https://api.twitch.tv/helix/chat/badges/global'

  response = requests.get(url, headers=headers)
  badges = response.json()

  df = pd.DataFrame(badges)

  display(df)

  print(badges["data"][0])

  # for badge in badges["data"]:
  #   # Put item into the table
  #   table.put_item(Item={
  #     'pk': badge['set_id'], 
  #     'sk': datetime.now().isoformat(), 
  #     'versions': badge['versions'], 
  #   })
else:
  # Handle errors
  print(f"Error: {response.status_code}, {response.text}")

In [ ]:
# Save all emotes to DynamoDB

import boto3
import pandas as pd
import requests
import os
from datetime import datetime

# Configure AWS credentials
aws_access_key_id = os.environ.get('AWS_ACCESS_KEY')
aws_secret_access_key = os.environ.get('AWS_SECRET_KEY')
aws_region = os.environ.get('AWS_REGION')
aws_table = os.environ.get('AWS_TABLE')

# Create a DynamoDB resource
dynamodb = boto3.resource(
  'dynamodb', 
  aws_access_key_id=aws_access_key_id,                        
  aws_secret_access_key=aws_secret_access_key,
  region_name=aws_region
)

# Configure Twitch credentials
client_id = os.environ.get('TWITCH_CLIENT_ID')
client_secret = os.environ.get('TWITCH_CLIENT_SECRET')
oauth = f"https://id.twitch.tv/oauth2/token?client_id={client_id}&client_secret={client_secret}&grant_type=client_credentials"

# Get the DynamoDB table
table = dynamodb.Table(aws_table)

# Make the API request
response = requests.post(oauth)

# Check the response
if response.status_code == 200:
  # Successful request

  # Set your environmental variable
  api_key = response.json()["access_token"]

  # Construct headers with the required information
  headers = {
    'Authorization': f'Bearer {api_key}',
    'Client-Id': client_id
  }

  # API endpoint URL
  url = 'https://api.twitch.tv/helix/chat/emotes/global'

  response = requests.get(url, headers=headers)
  emotes = response.json()

  df = pd.DataFrame(emotes)

  display(df)

  print(emotes["data"][0])

  # for emote in emotes["data"]:
  #   # Put item into the table
  #   table.put_item(Item={
  #     'pk': emote['id'], 
  #     'sk': datetime.now().isoformat(), 
  #     'name': emote['name'], 
  #     'images': emote['images'], 
  #     'format': emote['format'], 
  #     'scale': emote['scale'],
  #     'themes': emote['theme_mode'],
  #   })
else:
  # Handle errors
  print(f"Error: {response.status_code}, {response.text}")

In [ ]:
# Get top clips for the week

import boto3
import pandas as pd
import requests
import os
import uuid
from datetime import datetime, timedelta
from decimal import Decimal  # For handling Decimal types in DynamoDB
from urllib.parse import quote

# Configure Twitch credentials
client_id = os.environ.get('TWITCH_CLIENT_ID')
client_secret = os.environ.get('TWITCH_CLIENT_SECRET')
oauth = f"https://id.twitch.tv/oauth2/token?client_id={client_id}&client_secret={client_secret}&grant_type=client_credentials"

# Configure AWS credentials
aws_access_key_id = os.environ.get('AWS_ACCESS_KEY')
aws_secret_access_key = os.environ.get('AWS_SECRET_KEY')
aws_region = os.environ.get('AWS_REGION')
aws_table = os.environ.get('AWS_TABLE')

# Create a DynamoDB resource
dynamodb = boto3.resource(
  'dynamodb', 
  aws_access_key_id=aws_access_key_id,                        
  aws_secret_access_key=aws_secret_access_key,
  region_name=aws_region
)

# Get the DynamoDB table
table = dynamodb.Table(aws_table)

# Make the API request
response = requests.post(oauth)

# Check the response
if response.status_code == 200:
  # Successful request

  # Set your environmental variable
  api_key = response.json()["access_token"]

  # Construct headers with the required information
  headers = {
    'Authorization': f'Bearer {api_key}',
    'Client-Id': client_id
  }

  pagination = None
  all_games = list()
  amount = 50

  # API endpoint URL
  url = f'https://api.twitch.tv/helix/games/top?first={amount}'

  response = requests.get(url, headers=headers)
  games = response.json()

  all_games.extend(games["data"])

  pagination = None
  all_clips = list()
  amount = 100

  # Get today's date at 12:00 AM
  today = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)

  # Get last week's date at 12:00 AM
  last_week = today - timedelta(days=7)

  # Making strings compatible to Twitch's started_at and ended_at query parameters
  today = today.strftime('%Y-%m-%dT%H:%M:%SZ')
  last_week = last_week.strftime('%Y-%m-%dT%H:%M:%SZ')

  for game in games["data"]: 
    # API endpoint URL
    url = f'https://api.twitch.tv/helix/clips?is_featured={True}&started_at={last_week}&ended_at={today}&game_id={game["id"]}&first={amount}'

    print(game["id"])
    
    response = requests.get(url, headers=headers)
    clips = response.json()

    all_clips.extend(clips["data"])

  print(len(all_clips))

  # Sort the list of dictionaries based on 'view_count' field
  sorted_list = sorted(all_clips, key=lambda x: x['view_count'], reverse=True)

  # Get the top 500 elements
  top_500 = sorted_list[:500]

  # Print or use top 500 elements
  print(len(top_500))

  df = pd.DataFrame(top_500)

  display(df)

  # Save all clips to DynamoDB

  for clip in top_500:
    # Define PK and SK
    id = str(uuid.uuid4())
    pk = f'CLIP#{id}'
    sk = f'CLIP#{id}'
    
    # Convert clip object into DynamoDB item format
    item = {
      'pk': pk,
      'sk': sk,
      'id': clip['id'],
      'url': clip['url'],
      'embed_url': clip['embed_url'],
      'broadcaster_id': clip['broadcaster_id'],
      'broadcaster_name': clip['broadcaster_name'],
      'creator_id': clip['creator_id'],
      'creator_name': clip['creator_name'],
      'video_id': clip['video_id'],
      'game_id': clip['game_id'],
      'language': clip['language'],
      'title': clip['title'],
      'view_count': Decimal(clip['view_count']),  # Convert to Decimal for DynamoDB
      'created_at': clip['created_at'],
      'thumbnail_url': clip['thumbnail_url'],
    }
  
    # Put item into DynamoDB table
    table.put_item(Item=item)
  else:
    # Handle errors
    print(f"Error: {response.status_code}, {response.text}")

In [ ]:
# Get top live channels

import pandas as pd
import requests
import os
from datetime import datetime

# Configure Twitch credentials
client_id = os.environ.get('TWITCH_CLIENT_ID')
client_secret = os.environ.get('TWITCH_CLIENT_SECRET')
oauth = f"https://id.twitch.tv/oauth2/token?client_id={client_id}&client_secret={client_secret}&grant_type=client_credentials"

# Make the API request
response = requests.post(oauth)

# Check the response
if response.status_code == 200:
  # Successful request

  # Set your environmental variable
  api_key = response.json()["access_token"]

  # Construct headers with the required information
  headers = {"Authorization": f"Bearer {api_key}", "Client-Id": client_id }

  amount = 100        

  # API endpoint URL
  url = f"https://api.twitch.tv/helix/streams?first={amount}"

  response = requests.get(url, headers=headers)
  channels = response.json()

  df = pd.DataFrame(channels["data"])

  display(df)
else:
  # Handle errors
  print(f"Error: {response.status_code}, {response.text}")

In [ ]:
# Get channel data

import pandas as pd
import requests
import os

# Configure Twitch credentials
client_id = os.environ.get('TWITCH_CLIENT_ID')
client_secret = os.environ.get('TWITCH_CLIENT_SECRET')
oauth = f"https://id.twitch.tv/oauth2/token?client_id={client_id}&client_secret={client_secret}&grant_type=client_credentials"

# Make the API request
response = requests.post(oauth)

# Check the response
if response.status_code == 200:
  # Successful request

  # Set your environmental variable
  api_key = response.json()["access_token"]

  # Construct headers with the required information
  headers = {
    'Authorization': f'Bearer {api_key}',
    'Client-Id': client_id
  }

  pagination = None
  all_games = list()
  amount = 50
  user = "xqc"

  # API endpoint URL
  if user.isdigit():
    url = f'https://api.twitch.tv/helix/users?id={user}'
  else:
    url = f'https://api.twitch.tv/helix/users?login={user}'

  response = requests.get(url, headers=headers)
  channel = response.json()

  df = pd.DataFrame(channel["data"])

  display(df)

  login = channel["data"]

  url = f'https://twitchtracker.com/api/channels/summary/{user}'
  
  response = requests.get(url, headers=headers)
  statistics = response.json()

  print(statistics)

  df = pd.DataFrame([statistics])

  display(df)
else:
  # Handle errors
  print(f"Error: {response.status_code}, {response.text}")

In [ ]:
# Get streams based on parameters

import pandas as pd
import requests
import os

# Configure Twitch credentials
client_id = os.environ.get('TWITCH_CLIENT_ID')
client_secret = os.environ.get('TWITCH_CLIENT_SECRET')
oauth = f"https://id.twitch.tv/oauth2/token?client_id={client_id}&client_secret={client_secret}&grant_type=client_credentials"

# Make the API request
response = requests.post(oauth)

# Check the response
if response.status_code == 200:
  # Successful request

  # Set your environmental variable
  api_key = response.json()["access_token"]

  # Construct headers with the required information
  headers = {"Authorization": f"Bearer {api_key}", "Client-Id": client_id }

  pagination = None
  all_streams = list()
  amount = 100
  game = "33214"
  language = "en"
  viewers = 10
  user = "xqc"

  while True: 
    # API endpoint URL
    if pagination is None:
      url = f"https://api.twitch.tv/helix/streams?type=live&first={amount}"
    else:
      url = f"https://api.twitch.tv/helix/streams?type=live&first={amount}&after={pagination}"

    if game:
      url += f"&game_id={game}"
    if language:
      url += f"&language={language}"

    response = requests.get(url, headers=headers)
    streams = response.json()

    all_streams.extend(streams["data"])

    if not (streams["pagination"]) or len(all_streams) >= 500:
      break
    
    pagination = streams["pagination"]["cursor"]

    print(pagination)       

  df = pd.DataFrame(all_streams)

  display(df)
else:
  # Handle errors
  print(f"Error: {response.status_code}, {response.text}")